In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/digit-recognizer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# Import Libraries

In [2]:
from skimage.io import  imread, imshow
from skimage.transform import  resize, rescale
from skimage.color import rgb2gray
import numpy as np
from os import listdir, path
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import KFold

# Read the Data

In [3]:
data_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
data_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data_submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

# Look at the Data

In [4]:
data_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data_train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_submission

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


## Look at the shape 

In [7]:
data_test.shape

(28000, 784)

In [8]:
data_train.shape

(42000, 785)

In [9]:
data_submission.shape

(28000, 2)

# Splite the target column

## For Train Data

In [10]:
X_train = data_train.drop(columns=['label'])
X_train

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y_train = data_train['label']
y_train

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [12]:
X_train.shape

(42000, 784)

In [13]:
y_train.shape

(42000,)

## For Test Data

In [14]:
y_test = data_submission['Label']
y_test

0        0
1        0
2        0
3        0
4        0
        ..
27995    0
27996    0
27997    0
27998    0
27999    0
Name: Label, Length: 28000, dtype: int64

# Preprocessing (To Categorical)

In [15]:
y_train_k = to_categorical(y_train, 10)
y_train_k

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [16]:
y_train_k.shape

(42000, 10)

In [17]:
y_test_k = to_categorical(y_test, 10)
y_test_k

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
y_test_k.shape

(28000, 10)

# Deep Learning Model

In [19]:
import tensorflow as tf
from tensorflow import keras

In [20]:
#  Design Model

from keras import layers

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
# to make this notebook's output stable across runs
import tensorflow as tf
def reset_graph(seed=41):
    tf.random.set_seed(seed)
    np.random.seed(seed) 

In [23]:
reset_graph(seed=42)
tf.random.set_seed(500)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3) # Early Stoping

model = keras.Sequential(
    [  
        layers.InputLayer(input_shape=(784, )),
        layers.Dense(1000, activation="relu", name="hiddenlayer"),
        layers.Dense(900, activation="relu", name="hiddenlayer1"),
        layers.Dense(800, activation="relu", name="hiddenlayer2"),
        layers.Dense(700, activation="relu", name="hiddenlayer3"),
        layers.Dense(600, activation="relu", name="hiddenlayer4"),
        layers.Dense(500, activation="relu", name="hiddenlayer5"),
        layers.Dense(400, activation="relu", name="hiddenlayer6"),
        layers.Dense(300, activation="relu", name="hiddenlayer7"),
        layers.Dense(200, activation="relu", name="hiddenlayer8"),
        layers.Dense(100, activation="relu", name="hiddenlayer9"),
        layers.Dense(50, activation="relu", name="hiddenlayer10"),
        layers.Dense(10, activation="softmax", name="outputlayer"),
    ]
)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hiddenlayer (Dense)          (None, 1000)              785000    
_________________________________________________________________
hiddenlayer1 (Dense)         (None, 900)               900900    
_________________________________________________________________
hiddenlayer2 (Dense)         (None, 800)               720800    
_________________________________________________________________
hiddenlayer3 (Dense)         (None, 700)               560700    
_________________________________________________________________
hiddenlayer4 (Dense)         (None, 600)               420600    
_________________________________________________________________
hiddenlayer5 (Dense)         (None, 500)               300500    
_________________________________________________________________
hiddenlayer6 (Dense)         (None, 400)               2

In [25]:
reset_graph(seed=9)

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model_training = model.fit(X_train, y_train_k, epochs=40, callbacks=[callback])

Epoch 1/40
1313/1313 [==============================] - 8s 4ms/step - loss: 0.4537 - accuracy: 0.8908
Epoch 2/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.2403 - accuracy: 0.9483
Epoch 3/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1847 - accuracy: 0.9600
Epoch 4/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1518 - accuracy: 0.9676
Epoch 5/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1526 - accuracy: 0.9684
Epoch 6/40
1313/1313 [==============================] - 6s 5ms/step - loss: 0.1367 - accuracy: 0.9704
Epoch 7/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1438 - accuracy: 0.9706
Epoch 8/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1171 - accuracy: 0.9754
Epoch 9/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.1356 - accuracy: 0.9734
Epoch 10/40
1313/1313 [==============================] - 5s 4ms/step - loss: 0.175

In [26]:
#testing
test_pred = model.predict(data_test).argmax(axis=-1)

In [27]:
test_pred

array([2, 0, 9, ..., 3, 9, 2])

In [28]:
data_submission.Label = test_pred
data_submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [29]:
data_submission.to_csv("data_submission.csv", index=False, header=True)